# Age of Empires II Voobly Matches Data Exploration

## Load libraries and set options/environment

In [2]:
# System
import sys
import os
import datetime
from datetime import date, timedelta
sys.path.append('C:/Users/Parikshit_verma/Documents/GitHub/AgeofEmpires2_Voobly/src/functions')

# Custom functions
from setup_sql_conn import setup_sql_conn,fetch_sql_data
from sql_read import sql_read

# Data Wrangling
import pandas as pd
import numpy as np
import re
from functools import reduce
from pandas_profiling import ProfileReport

# Visualization
import seaborn as sb
import matplotlib.pyplot as plt
import plotly

# Set options
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Read Data

In [48]:
db_conn,db_cursor,engine = setup_sql_conn('localhost','AOE2_VOOBLY')
match_data = sql_read('C:/Users/Parikshit_verma/Documents/GitHub/AgeofEmpires2_Voobly/Input/match_data_query.sql',db_conn)

Time taken to read from server : 3.835047483444214


## Data Mining & Profiling

In [52]:
# Match data cleanup
# Civilization dictionary
civ_map = ['Aztecs','Berbers','Britons','Bulgarians','Burmese','Byzantines','Celts','Chinese','Cumans','Ethiopians','Franks','Goths','Huns','Incas','Indians'
            ,'Italians','Japanese','Khmer','Koreans','Lithuanians','Magyars','Malay','Malians','Mayans','Mongols','Persians','Portuguese','Saracens','Slavs'
            ,'Spanish','Tatars','Teutons','Turks','Vietnamese','Vikings']
civ_map = {i.upper()[0:3] : i for i in civ_map}

match_data = (match_data
              .assign(Player_Rating = match_data['New_Rating'] - match_data['Points'],
                      Civilization = match_data['Civilization'].map(civ_map),
                      Match_date = pd.to_datetime(match_data['Match_date']),
                      Match_time = pd.to_datetime(match_data['Match_time'], format='%H:%M:%S').dt.hour)
              .drop(['Match_rating','Match_player_no','New_Rating','ladder',
                     'Trade_Profit','Tribute_Received','Tribute_Sent',
                     'Team'],axis=1)
              .dropna(subset=['Civilization']))

In [57]:
match_data

,Match_ID,Match_date,Match_time,Match_map,Match_length,Match_mod,Player,Points,Team,Civilization,Victory,Military_Score,Economy_Score,Technology_Score,Society_Score,Total_Score,Units_Killed,Units_Lost,Buildings_Razed,Buildings_Lost,Units_Converted,Food,Wood,Stone,Gold,Tribute_Received,Tribute_Sent,Feudal_Time,Castle_Time,Imperial_Time,Map_Explored,Research_Count,Research_Percent,Total_Wonders,Total_Castles,Relics_Captured,Relic_Gold,Villager_High,Player_Rating
0,20850764,2019-11-02,20,Black Forest,12,v1.5 RC | Longer Lasting Resou,yuriiarabchuk,-4,2,Celts,0,0,301,1010,0,1311,0,2,0,0,0,1278,0,0,0,0,0,None,None,None,1.00,1,0.01,0,0,0,0,8,1387
1,20394127,2019-08-17,0,Arabia,6,v1.5 RC | WololoKingdoms,[AML]MiguelBayona,-17,2,Goths,0,0,295,190,0,485,0,1,0,0,0,374,280,0,0,0,0,None,None,None,0.18,1,0.00,0,0,0,0,14,1268
2,20394168,2019-08-17,0,Arabia,9,v1.5 RC | WololoKingdoms,Kensema,-22,2,Magyars,0,0,411,280,0,691,0,2,0,0,0,1098,371,0,0,0,0,None,None,None,0.27,1,0.00,0,0,0,0,21,1394
3,20394169,2019-08-17,0,Custom,9,v1.5 Beta R7,echoccccc,16,1,Chinese,1,0,463,310,0,773,0,0,0,0,0,1386,576,0,0,0,0,None,None,None,0.30,1,0.00,0,0,0,0,23,1558
4,20394169,2019-08-17,0,Custom,9,v1.5 Beta R7,Nemo_Fulani,-16,2,Saracens,0,0,503,340,0,843,0,0,0,0,0,1289,540,0,0,0,0,None,None,None,0.34,0,0.00,0,0,0,0,23,1550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,21162234,2020-01-09,0,Arabia,56,v1.5 Beta R7 | WololoKingdoms,[TEU]miza2193,-12,2,Portuguese,0,1927,1095,1141,390,4553,79,156,6,12,3,9877,11044,2447,4394,0,0,12,27,None,0.41,18,0.11,2,3,0,0,73,1296
99996,21167996,2020-01-10,13,Arabia,40,v1.5 RC | WololoKingdoms,_UK_rainbow,-17,2,Franks,0,1010,2820,1544,97,5471,49,66,0,7,4,12886,11372,1592,4186,0,0,10,20,None,0.64,22,0.15,0,1,0,0,89,1898
99997,21217342,2020-01-22,1,Arabia,60,v1.5 RC | WololoKingdoms,WhoCl,-19,2,Goths,0,4553,2515,1481,390,8939,199,244,1,14,7,16471,16844,3149,5374,0,0,9,27,None,0.61,20,0.13,1,3,0,0,101,2070
99998,21235713,2020-01-26,15,Black Forest,29,v1.5 RC | Longer Lasting Resou,Lucio_Nunes_Jr,-16,2,Franks,0,302,1385,1450,97,3234,11,23,0,1,1,5149,4538,840,1439,0,0,10,17,None,1.00,11,0.11,0,1,0,0,45,1662


In [70]:
match_data_max = (match_data
                  .groupby(['Match_ID'])['Military_Score','Economy_Score','Technology_Score','Society_Score','Total_Score',
                                         'Units_Killed','Units_Lost','Buildings_Razed','Buildings_Lost','Units_Converted',
                                         'Food','Wood','Stone','Gold',
                                         'Map_Explored','Research_Count','Research_Percent',
                                         'Total_Wonders','Total_Castles','Relics_Captured','Relic_Gold','Villager_High',
                                         'Player_Rating'].max()
                  .add_suffix('_max')).reset_index()

match_data_min = (match_data
                  .groupby(['Match_ID'])['Military_Score','Economy_Score','Technology_Score','Society_Score','Total_Score',
                                         'Units_Killed','Units_Lost','Buildings_Razed','Buildings_Lost','Units_Converted',
                                         'Food','Wood','Stone','Gold',
                                         'Map_Explored','Research_Count','Research_Percent',
                                         'Total_Wonders','Total_Castles','Relics_Captured','Relic_Gold','Villager_High',
                                         'Player_Rating'].min()
                  .add_suffix('_min')).reset_index()

In [68]:
match_data_max

,Match_ID,Military_Score_max,Economy_Score_max,Technology_Score_max,Society_Score_max,Total_Score_max,Units_Killed_max,Units_Lost_max,Buildings_Razed_max,Buildings_Lost_max,Units_Converted_max,Food_max,Wood_max,Stone_max,Gold_max,Map_Explored_max,Research_Count_max,Research_Percent_max,Total_Wonders_max,Total_Castles_max,Relics_Captured_max,Relic_Gold_max,Villager_High_max,Player_Rating_max
0,20001122,1025,2512,1862,390,5789,38,77,4,5,0,11889,12641,2721,4886,0.62,19,0.18,3,3,0,0,77,1317
1,20001249,4675,2667,2677,520,10539,302,208,0,27,0,28519,21426,4489,12504,0.84,30,0.28,12,4,0,0,104,1481
2,20001907,677,1628,1400,0,3705,46,62,0,6,0,8427,7208,207,1514,0.70,15,0.09,0,0,0,0,43,1606
3,20002228,483,3046,1672,0,5201,27,71,0,3,0,11253,12200,1080,3886,0.72,22,0.14,0,0,0,0,97,1592
4,20002531,1233,2480,2165,390,6268,42,30,22,1,0,6629,8763,4193,5907,1.00,14,0.14,11,3,0,0,78,1340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60145,21309325,289,1870,1085,260,3474,24,24,0,0,0,5866,5402,1677,2001,0.53,12,0.08,0,2,0,0,64,1697
60146,21309329,20,1046,955,0,1699,2,2,0,0,0,2902,2444,0,628,0.60,8,0.05,0,0,0,0,38,1528
60147,21309330,134,1217,678,0,2029,13,13,0,0,0,3641,3072,0,890,0.46,6,0.04,0,0,0,0,40,1572
60148,21309333,610,2640,1563,0,4813,44,44,3,3,0,9400,7565,754,3651,0.68,22,0.14,0,0,0,0,89,1688


In [73]:
match_data_min[match_data_min['Match_ID']==20001122]

,Match_ID,Military_Score_min,Economy_Score_min,Technology_Score_min,Society_Score_min,Total_Score_min,Units_Killed_min,Units_Lost_min,Buildings_Razed_min,Buildings_Lost_min,Units_Converted_min,Food_min,Wood_min,Stone_min,Gold_min,Map_Explored_min,Research_Count_min,Research_Percent_min,Total_Wonders_min,Total_Castles_min,Relics_Captured_min,Relic_Gold_min,Villager_High_min,Player_Rating_min
0,20001122,1025.0,2512.0,1862.0,390.0,5789.0,38.0,77.0,4.0,5.0,0.0,11889.0,12641.0,2721.0,4886.0,0.62,19.0,0.18,3.0,3.0,0.0,0.0,77.0,1317.0


In [74]:
match_data[match_data['Match_ID']==20001122]

,Match_ID,Match_date,Match_time,Match_map,Match_length,Match_mod,Player,Points,Team,Civilization,Victory,Military_Score,Economy_Score,Technology_Score,Society_Score,Total_Score,Units_Killed,Units_Lost,Buildings_Razed,Buildings_Lost,Units_Converted,Food,Wood,Stone,Gold,Tribute_Received,Tribute_Sent,Feudal_Time,Castle_Time,Imperial_Time,Map_Explored,Research_Count,Research_Percent,Total_Wonders,Total_Castles,Relics_Captured,Relic_Gold,Villager_High,Player_Rating
70666,20001122,2019-06-11,5,Arabia,45,v1.5 Beta R7,[CLB]LeonidasXIX,-11,2,Britons,0,1025,2512,1862,390,5789,38,77,4,5,0,11889,12641,2721,4886,0,0,11,25,40,0.62,19,0.18,3,3,0,0,77,1317
